In [1]:
#n.p check input at native or standard, check mask as native or standard, check time series, check parameters too of course

In [ ]:
#first and foremost - no more changing
import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image
from nilearn.glm.first_level import compute_regressor
import logging
from brainiak.searchlight.searchlight import Searchlight, Ball

from statsmodels.tsa.stattools import grangercausalitytests
from scipy import stats
import sys
from mpi4py import MPI
import gc
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
import sys
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Scramble'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
subs = sub_info['sub'].tolist()
subs = ['sub-025']

run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# Searchlight parameters
searchlight_radius = 2  # in voxels, adjust as needed
max_blk_edge = 10
pool_size = 1

def load_and_prepare_data(sub, run_combo):
    """
    Load and prepare data for a single subject and run combination.
    
    :param sub: Subject ID
    :param run_combo: List of run numbers to combine
    :return: Tuple of (4D fMRI data, brain mask, psychological covariate)
    """
    logging.info(f"Loading data for subject {sub}, runs {run_combo}")
    
    # Load and combine run data
    run_data_list = []
    for run in run_combo:
        run_file = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz' #registered but not standardaized
        run_img = image.load_img(run_file)
        run_data = image.clean_img(run_img, standardize=True)
        run_data_list.append(run_data)
    
    # Concatenate run data
    fmri_data = image.concat_imgs(run_data_list)
    
    # Load brain mask
    mask_file = f'{study_dir}/{sub}/ses-01/derivatives/rois/parcels/pIPS.nii.gz'
    brain_mask = nib.load(mask_file).get_fdata().astype(bool)
    
    # Generate psychological covariate
    psy = make_psy_cov(run_combo, sub)
    
    # Ensure fMRI data and psy covariate have the same number of time points
    if fmri_data.shape[-1] != len(psy):
        raise ValueError(f"Mismatch in volumes: fMRI data has {fmri_data.shape[-1]}, psy has {len(psy)}")
    
    # Convert fmri_data to 4D numpy array if it's not already
    if isinstance(fmri_data, nib.Nifti1Image):
        fmri_data = fmri_data.get_fdata()
    
    # Ensure fmri_data is 4D
    if fmri_data.ndim != 4:
        raise ValueError(f"fMRI data must be 4D, but got shape {fmri_data.shape}")
    
    return fmri_data, brain_mask, psy

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols_per_run, tr = 184, 2.0
    total_vols = vols_per_run * len(runs)
    times = np.arange(0, total_vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * vols_per_run * tr
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy


# Global variable declaration
global psy

def gca_measure(data, mask, myrad, bcvar):
    global psy
    
    data_4d = data[0]
    data_2d = data_4d.reshape(data_4d.shape[0], -1)
    center_ts = data_2d[:, 0]
    other_ts = np.mean(data_2d[:, 1:], axis=1)
    
    gc_center_to_other = grangercausalitytests(np.column_stack((other_ts, center_ts, psy.flatten())), maxlag=1, verbose=False)
    gc_other_to_center = grangercausalitytests(np.column_stack((center_ts, other_ts, psy.flatten())), maxlag=1, verbose=False)
    
    f_diff = gc_center_to_other[1][0]['ssr_ftest'][0] - gc_other_to_center[1][0]['ssr_ftest'][0]
    
    return f_diff

def run_searchlight(fmri_data, brain_mask):
    assert brain_mask.ndim == 3 and brain_mask.dtype == bool, "Invalid brain_mask"

    sl = Searchlight(sl_rad=searchlight_radius, max_blk_edge=10)
    fmri_data_4d = fmri_data.transpose(3, 0, 1, 2)
    fmri_data_list = [fmri_data_4d]
    
    sl.distribute(fmri_data_list, brain_mask)
    sl_result = sl.run_searchlight(gca_measure, pool_size=1)
    
    return sl_result

if __name__ == "__main__":
    global psy
    
    for sub in subs:
        for run_combo in run_combos:
            try:
                fmri_data, brain_mask, psy = load_and_prepare_data(sub, run_combo)
                print(f"Processing subject {sub}, runs {run_combo}")
                print(f"Data shapes - fMRI: {fmri_data.shape}, Mask: {brain_mask.shape}, PSY: {psy.shape}")
                
                assert psy.shape[0] == fmri_data.shape[3], "PSY shape mismatch"
                
                sl_result = run_searchlight(fmri_data, brain_mask)
                
                result_file = f'{results_dir}/searchlight_gca_{sub}_runs{"_".join(map(str, run_combo))}.nii.gz'
                nib.save(nib.Nifti1Image(sl_result, affine=nib.load(f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-01/1stLevel.feat/filtered_func_data_reg.nii.gz').affine), result_file)
                
                print(f"Completed analysis for subject {sub}, runs {run_combo}")
                
            except AssertionError as e:
                print(f"Assertion error for subject {sub}, runs {run_combo}: {str(e)}")
            except Exception as e:
                print(f"Error processing subject {sub}, runs {run_combo}: {str(e)}")

In [ ]:
# take 2 gpt

import os
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image
from nilearn.glm.first_level import compute_regressor
import logging
from brainiak.searchlight.searchlight import Searchlight
from statsmodels.tsa.stattools import grangercausalitytests
import sys
from mpi4py import MPI
import gc
import time

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = '/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
localizer = 'Scramble'  # scramble or object. This is the localizer task.
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
sub_info = sub_info[sub_info['group'] == 'control']
#subs = sub_info['sub'].tolist()
# For testing, you can uncomment the line below to include all subjects
subs = ['sub-025']

run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

# Searchlight parameters
searchlight_radius = 2  # in voxels, adjust as needed
max_blk_edge = 10
pool_size = 1

# Constants
VOL_PER_RUN = 184
TR = 2.0

def load_and_prepare_data(sub, run_combo):
    """
    Load and prepare data for a single subject and run combination.
    
    :param sub: Subject ID
    :param run_combo: List of run numbers to combine
    :return: Tuple of (4D fMRI data, brain mask, psychological covariate)
    """
    logging.info(f"Loading data for subject {sub}, runs {run_combo}")

    # Load and combine run data
    run_data_list = []
    for run in run_combo:
        run_file = f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-0{run}/1stLevel.feat/filtered_func_data_reg.nii.gz'
        run_img = image.load_img(run_file)
        run_data = image.clean_img(run_img, standardize=True)
        run_data_list.append(run_data)
    
    # Concatenate run data
    fmri_data = image.concat_imgs(run_data_list)
    
    # Load brain mask
    mask_file = f'{study_dir}/{sub}/ses-01/derivatives/rois/parcels/pIPS.nii.gz'
    brain_mask = nib.load(mask_file).get_fdata().astype(bool)

    # Generate psychological covariate
    psy = make_psy_cov(run_combo, sub)

    # Ensure fMRI data and psy covariate have the same number of time points
    if fmri_data.shape[-1] != len(psy):
        raise ValueError(f"Mismatch in volumes: fMRI data has {fmri_data.shape[-1]}, psy has {len(psy)}")
    
    # Convert fmri_data to 4D numpy array if it's not already
    if isinstance(fmri_data, nib.Nifti1Image):
        fmri_data = fmri_data.get_fdata()
    
    # Ensure fmri_data is 4D
    if fmri_data.ndim != 4:
        raise ValueError(f"fMRI data must be 4D, but got shape {fmri_data.shape}")
    
    return fmri_data, brain_mask, psy

def make_psy_cov(runs, ss):
    """
    Create psychological covariate data for the specified runs and subject.

    :param runs: List of run numbers
    :param ss: Subject ID
    :return: Psychological covariate as a numpy array
    """
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    total_vols = VOL_PER_RUN * len(runs)
    times = np.arange(0, total_vols * TR, TR)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for i, rn in enumerate(runs):
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_{localizer}.txt'

        if not os.path.exists(obj_cov_file):
            logging.warning(f'Covariate file not found for run {rn}')
            return np.zeros((total_vols, 1))  # Return a zeros array if file not found

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        
        if i > 0:
            obj_cov['onset'] += i * VOL_PER_RUN * TR
        
        full_cov = pd.concat([full_cov, obj_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        logging.warning('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((total_vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    psy[psy > 0] = 1
    psy[psy <= 0] = 0
    return psy

def gca_measure(data, mask, myrad, psy):
    """
    Measure Granger causality between the center and surrounding time series.

    :param data: 4D fMRI data
    :param mask: Brain mask
    :param myrad: Searchlight radius
    :param psy: Psychological covariate
    :return: Difference in Granger causality measures
    """
    data_4d = data[0]
    data_2d = data_4d.reshape(data_4d.shape[0], -1)
    center_ts = data_2d[:, 0]
    other_ts = np.mean(data_2d[:, 1:], axis=1)
    
    gc_center_to_other = grangercausalitytests(np.column_stack((other_ts, center_ts, psy.flatten())), maxlag=1, verbose=False)
    gc_other_to_center = grangercausalitytests(np.column_stack((center_ts, other_ts, psy.flatten())), maxlag=1, verbose=False)
    
    f_diff = gc_center_to_other[1][0]['ssr_ftest'][0] - gc_other_to_center[1][0]['ssr_ftest'][0]
    
    return f_diff

def run_searchlight(fmri_data, brain_mask, psy):
    """
    Run searchlight analysis on the fMRI data.

    :param fmri_data: 4D fMRI data
    :param brain_mask: Brain mask
    :param psy: Psychological covariate
    :return: Searchlight results
    """
    assert brain_mask.ndim == 3 and brain_mask.dtype == bool, "Invalid brain_mask"
    assert fmri_data.shape[0:3] == brain_mask.shape, "Brain mask dimensions do not match fMRI data."

    sl = Searchlight(sl_rad=searchlight_radius, max_blk_edge=max_blk_edge)
    fmri_data_4d = fmri_data.transpose(3, 0, 1, 2)
    fmri_data_list = [fmri_data_4d]
    
    sl.distribute(fmri_data_list, brain_mask)
    sl_result = sl.run_searchlight(gca_measure, psy=psy, pool_size=pool_size)
    
    return sl_result

if __name__ == "__main__":
    for sub in subs:
        for run_combo in run_combos:
            try:
                fmri_data, brain_mask, psy = load_and_prepare_data(sub, run_combo)
                logging.info(f"Processing subject {sub}, runs {run_combo}")
                logging.info(f"Data shapes - fMRI: {fmri_data.shape}, Mask: {brain_mask.shape}, PSY: {psy.shape}")
                
                assert psy.shape[0] == fmri_data.shape[3], "PSY shape mismatch"
                
                sl_result = run_searchlight(fmri_data, brain_mask, psy)
                
                result_file = f'{results_dir}/searchlight_gca_{sub}_runs{"_".join(map(str, run_combo))}.nii.gz'
                affine = nib.load(f'{raw_dir}/{sub}/ses-01/derivatives/fsl/loc/run-01/1stLevel.feat/filtered_func_data_reg.nii.gz').affine
                nib.save(nib.Nifti1Image(sl_result, affine), result_file)
                
                logging.info(f"Completed analysis for subject {sub}, runs {run_combo}")
                
            except AssertionError as e:
                logging.error(f"Assertion error for subject {sub}, runs {run_combo}: {str(e)}")
            except Exception as e:
                logging.error(f"Error processing subject {sub}, runs {run_combo}: {str(e)}")
            finally:
                gc.collect()


/home/csimmon2/anaconda3/envs/brainiak_env/lib/python3.7/site-packages/nilearn/__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()
2024-10-07 16:55:53,267 - INFO - Loading data for subject sub-025, runs [1, 2]
